In [41]:
class Solution:
    def calculate(self, s: str, use_int: bool = True) -> int:

        expr = s.replace(" ", "")
        pos = 0
        n = len(expr)

        def peek():
            nonlocal pos
            if pos < n:
                return expr[pos]
            return None
        
        def consume():
            nonlocal pos
            char = peek()
            pos += 1
            return char
        
        def parse_number():
            nonlocal pos, expr
            start = pos
            if peek() == "-":
                consume()
            
            while peek() and peek().isdigit():
                consume()

            if peek() == ".":
                consume()
            
            while peek() and peek().isdigit():
                consume()

            num = expr[start:pos]
            if use_int:
                return int(float(num))
            return float(num)
        
        def parse_factor():
            """
            factor -> number | '(' expression ')' | '-' factor
            a number or expression in parentheses or -factor
            """
            # print("\tenter parse_factor")

            # unary minus
            if peek() == '-':
                consume()
                return -parse_factor()
            
            # parentheses
            if peek() == '(':
                consume()
                result = parse_expression()
                if peek() == ")":
                    consume()
                else:
                    raise ValueError("missing closing parenthesis")
                return result
            
            # number
            return parse_number()

        def parse_term():
            """
            term -> factor (('*' | '/') factor)*
            factor followed by zero or more occurances of *|/ then another factor
            """
            # print("\tenter parse_term")

            result = parse_factor()

            while(peek() in ("*", "/")):
                op = consume()
                right = parse_factor()

                if op == "*":
                    result *= right
                else:
                    if right == 0:
                        raise ValueError("division by zero")
                    result /= right
                    if use_int:
                        result = int(result)
            
            return result
        
        def parse_expression():
            nonlocal pos
            """
            expression -> term (('+' | "-") term)*
            term followed by zero or more +|- then another term
            """
            # print("\tenter parse_expression")

            result = parse_term()
            # print(f"result: {result}")
            while peek() in ("+", "-"):
                op = consume()
                right = parse_term()
                # print(f"right: {right}")
                if op == "+":
                    result += right
                else:
                    result -= right
            
            return result
        
        result = parse_expression()
        if pos < n:
            raise ValueError(f"unexpected character at position {pos}")
        
        return result
        

In [46]:
soln = Solution()

s = "3+2*2"
result = soln.calculate(s)
expected = 7
print(f"s: {s}, result: {result}", f", expected: {expected}, equal: {result == expected}")

s = " 3/2 "
result = soln.calculate(s)
expected = 1
print(f"s: {s}, result: {result}", f", expected: {expected}, equal: {result == expected}")

s = " 3+5 / 2 "
result = soln.calculate(s)
expected = 5
print(f"s: {s}, result: {result}", f", expected: {expected}, equal: {result == expected}")

s: 3+2*2, result: 7 , expected: 7, equal: True
s:  3/2 , result: 1 , expected: 1, equal: True
s:  3+5 / 2 , result: 5 , expected: 5, equal: True
